<a href="https://colab.research.google.com/github/kevinscaria/InstructABSA/blob/main/JointTask_Training_%26_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [1]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [2]:
if IN_COLAB:
  !pip install transformers
  !pip install datasets
  !pip install evaluate
  !pip install sentencepiece
  !pip install transformers[torch]
  !pip install peft

In [3]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/InstructABSA'
else:
    root_path = 'Enter local path'

use_mps = True if torch.has_mps else False
os.chdir(root_path)

<ipython-input-3-925b459844e6>:9: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  use_mps = True if torch.has_mps else False


In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.data_prep import DatasetLoader
from InstructABSA.utils import T5Generator, T5Classifier
from instructions import InstructionsHandler

In [5]:
class T5GeneratorWithLoRA(T5Generator):
    def __init__(self, model_checkpoint, lora_config):
        # Initialize the base T5Generator
        super().__init__(model_checkpoint)

        # Apply LoRA to the T5 model
        self.model = LoraModel(self.model, **lora_config)

## Training

In [6]:
####################warning, this tab is used for training a model that previously have been trained on task ATE
task_name = 'joint_task_flan-t5-base-instructiongen-ATE-trained'
# task_name = 'joint_task'
experiment_name = 'lapt2014_iabsa1'
# model_checkpoint = 'allenai/tk-instruct-base-def-pos'
# model_checkpoint = 'google/flan-t5-base'
model_checkpoint = '/content/drive/MyDrive/InstructABSA/Models/ate/pszemrajflan-t5-base-instructiongen-lapt2014_iabsa1_flant5_instruct_base'
print('Experiment Name: ', experiment_name)
model_out_path = './Models'
model_out_path = os.path.join(model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}")
print('Model output path: ', model_out_path)

Experiment Name:  lapt2014_iabsa1
Model output path:  ./Models/joint_task_flan-t5-base-instructiongen-ATE-trained/contentdriveMyDriveInstructABSAModelsatepszemrajflan-t5-base-instructiongen-lapt2014_iabsa1_flant5_instruct_base-lapt2014_iabsa1


In [ ]:
task_name = 'joint_task_flan-t5-small-instructiongen'
# task_name = 'joint_task'
experiment_name = 'lapt2014_iabsa1'
# model_checkpoint = 'allenai/tk-instruct-base-def-pos'
# model_checkpoint = 'google/flan-t5-base'
model_checkpoint = 'pszemraj/flan-t5-small-instructiongen'
print('Experiment Name: ', experiment_name)
model_out_path = './Models'
model_out_path = os.path.join(model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}")
print('Model output path: ', model_out_path)

Experiment Name:  lapt2014_iabsa1
Model output path:  ./Models/joint_task_flan-t5-small-instructiongen/pszemrajflan-t5-small-instructiongen-lapt2014_iabsa1


In [7]:
from sklearn.model_selection import train_test_split

our prompt

In [ ]:
aspe={}
aspe['bos_instruct1'] = """Definition: The output will be the aspects (both implicit and explicit) and the aspects sentiment polarity. In cases where there are no aspects the output should be noaspectterm:none.
        Positive example 1-
        input: I charge it at night and skip taking the cord with me because of the good battery life.
        output: battery life:positive,
        Positive example 2-
        input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!.
        output: features:positive, iChat:positive, Photobooth:positive, garage band:positive,

        Negative example 1-
        input: Startup times are incredibly long: over two minutes.
        output: Startup times:negative,
        Negative example 2-
        input: I had the same reasons as most PC users: the price, the overbearing restrictions of OSX and lack of support for games.
        output: price:negative, OSX:negative, support for games:negative,

        Neutral example 1-
        input: I am still in the process of learning about its features.
        output: features:Neutral,
        Neutral example 2-
        input: Took a long time trying to decide between one with retina display and one without.
        output: retina display:negative,
        Now complete the following example-

        input: """

aspe['eos_instruct'] = ' \noutput:'

original prompt

In [8]:
aspe={}
aspe['bos_instruct1'] = """Definition: The output will be the aspects (both implicit and explicit) and the aspects sentiment polarity. In cases where there are no aspects the output should be noaspectterm:none.
        Positive example 1-
        input: I charge it at night and skip taking the cord with me because of the good battery life.
        output: battery life:positive,
        Positive example 2-
        input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!.
        output: features:positive, iChat:positive, Photobooth:positive, garage band:positive
        Now complete the following example-
        input: """

aspe['eos_instruct'] = ' \noutput:'

In [9]:
# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

train_df,eval_df = train_test_split(id_tr_df,test_size=0.2,random_state=42)
# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df,eval_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_aspe_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', aspe['bos_instruct1'], aspe['eos_instruct'])
if loader.val_df_id is not None:
    loader.val_df_id = loader.create_data_in_aspe_format(loader.val_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', aspe['bos_instruct1'], aspe['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_aspe_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', aspe['bos_instruct1'], aspe['eos_instruct'])

#LORA

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
from peft import LoraConfig
from peft import LoraModel
target_modules = ["q", "v"]
lora_config = LoraConfig(
r=16,
target_modules = target_modules,
lora_alpha=64,
lora_dropout=0.05,
bias="none",
task_type="CAUSAL_LM",)

In [ ]:
from peft import get_peft_model

In [ ]:
# Create T5 utils object
t5_exp = T5Generator(model_checkpoint)

# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'lr_scheduler_type':'cosine',
    'per_device_train_batch_size':8,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/609 [00:00<?, ? examples/s]

In [ ]:
print(t5_exp.model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
t5_exp.model = get_peft_model(t5_exp.model, lora_config)
t5_exp.model.print_trainable_parameters()

trainable params: 1,769,472 || all params: 249,347,328 || trainable%: 0.7096414524241463


#Normal

In [10]:
# Create T5 utils object
t5_exp = T5Generator(model_checkpoint)

# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'lr_scheduler_type':'cosine',
    'per_device_train_batch_size':8,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/609 [00:00<?, ? examples/s]

In [11]:
# Train model
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.150981
2,0.275900,0.088407
3,0.145500,0.070684
4,0.112600,0.067111


## Inference

In [12]:
# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_aspe_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.aspe['bos_instruct1'], instruct_handler.aspe['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_aspe_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.aspe['bos_instruct1'], instruct_handler.aspe['eos_instruct'])

In [ ]:
t5_exp.model = t5_exp.model.merge_and_unload() #use for LoRa only

In [13]:
# Model inference - Loading from Checkpoint
# t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Model loaded to:  cuda


100%|██████████| 191/191 [01:52<00:00,  1.70it/s]


Model loaded to:  cuda


100%|██████████| 50/50 [00:29<00:00,  1.71it/s]


#Flan T5 Small without our prompt




In [ ]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Train Precision:  0.7731229597388466
Train Recall:  0.725925925925926
Train F1:  0.7487814517191411
Test Precision:  0.7242083758937692
Test Recall:  0.687015503875969
Test F1:  0.7051218299353557


In [ ]:
original_train_precision=0.842
original_train_recall=0.837
original_train_f1=0.8397
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)

(p/original_train_precision),(r/original_train_recall),(f1/original_train_f1)


(0.9181982894760649, 0.8672950130536751, 0.8917249633430286)

#Flan T5 Base

In [ ]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Train Precision:  0.8421591199795344
Train Recall:  0.8408684546615581
Train F1:  0.8415132924335379
Test Precision:  0.7960975609756098
Test Recall:  0.7906976744186046
Test F1:  0.7933884297520661


#Flan T5 Small with our prompt


In [ ]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Train Precision:  0.7401404646137223
Train Recall:  0.6998722860791826
Train F1:  0.71944335040042
Test Precision:  0.6945010183299389
Test Recall:  0.6608527131782945
Test F1:  0.6772591857000992


#Flan T5 Base with LoRa r=16, lora_alpha=64

In [ ]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)
print("LoRa config is: ",lora_config)

Train Precision:  0.7257975797579758
Train Recall:  0.674074074074074
Train F1:  0.6989802675142365
Test Precision:  0.7078059071729957
Test Recall:  0.6501937984496124
Test F1:  0.6777777777777777
LoRa config is:  LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='google/flan-t5-base', revision=None, task_type='CAUSAL_LM', inference_mode=False, r=16, target_modules={'q', 'v'}, lora_alpha=64, lora_dropout=0.01, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})


#Flan T5 Base with LoRa r=16, lora_alpha=32

In [ ]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)
print("LoRa config is: ",lora_config)

Train Precision:  0.6865168539325842
Train Recall:  0.6242656449553001
Train F1:  0.6539130434782608
Test Precision:  0.6538461538461539
Test Recall:  0.5930232558139535
Test F1:  0.6219512195121951
LoRa config is:  LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='google/flan-t5-base', revision=None, task_type='CAUSAL_LM', inference_mode=False, r=16, target_modules={'q', 'v'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})


#Flan T5 Base with LoRa r=8, lora_alpha=64

In [ ]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)
print("LoRa config is: ",lora_config)

Train Precision:  0.7193512919186366
Train Recall:  0.6684546615581098
Train F1:  0.6929696809214881
Test Precision:  0.7056962025316456
Test Recall:  0.6482558139534884
Test F1:  0.6757575757575758
LoRa config is:  LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='google/flan-t5-base', revision=None, task_type='CAUSAL_LM', inference_mode=False, r=8, target_modules={'q', 'v'}, lora_alpha=64, lora_dropout=0.01, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})


#Flan T5 Base with LoRa r=32, lora_alpha=64

In [ ]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)
print("LoRa config is: ",lora_config)

Train Precision:  0.7213249384067889
Train Recall:  0.6730523627075351
Train F1:  0.696353065539112
Test Precision:  0.7044025157232704
Test Recall:  0.6511627906976745
Test F1:  0.6767371601208458
LoRa config is:  LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='google/flan-t5-base', revision=None, task_type='CAUSAL_LM', inference_mode=False, r=32, target_modules={'q', 'v'}, lora_alpha=64, lora_dropout=0.01, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})


#Flan T5 Base with LoRa r=16, lora_alpha=64, lora_dropout=0.05

In [ ]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)
print("LoRa config is: ",lora_config)

Train Precision:  0.720885729907053
Train Recall:  0.6735632183908046
Train F1:  0.6964214974250627
Test Precision:  0.7021052631578948
Test Recall:  0.6463178294573644
Test F1:  0.6730575176589304
LoRa config is:  LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='google/flan-t5-base', revision=None, task_type='CAUSAL_LM', inference_mode=False, r=16, target_modules={'q', 'v'}, lora_alpha=64, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})


#flan-t5-base-instructiongen

In [ ]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Train Precision:  0.8653945332645694
Train Recall:  0.8572158365261814
Train F1:  0.8612857692801231
Test Precision:  0.8019512195121952
Test Recall:  0.7965116279069767
Test F1:  0.7992221682061255


#flan-t5-small-instructiongen

In [ ]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Train Precision:  0.7692722371967655
Train Recall:  0.7289910600255428
Train F1:  0.7485901639344263
Test Precision:  0.7195121951219512
Test Recall:  0.686046511627907
Test F1:  0.7023809523809523


#Flan T5 base instructiongen migrated(previously trained on ATE)

In [14]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Train Precision:  0.8902877697841727
Train Recall:  0.8850574712643678
Train F1:  0.8876649161009352
Test Precision:  0.8116504854368932
Test Recall:  0.810077519379845
Test F1:  0.8108632395732299
